# Module 2 Assignment: Wine Classification

In this notebook we will revisit our binary classification problem, but this time we will be classifying a real world dataset!

The dataset we have chosen for this assignment is the wine quality dataset (https://archive.ics.uci.edu/ml/datasets/wine+quality). These datasets include information on over 6000 bottles of red and white wine. Your task is to develop a Single Neuron Classifier that can discern between white and red wine with a reasonable accuracy. We have provided code below for assistance with uploading the files and preparing the dataset (this is a good chance for you to learn more Python by example). Additionally we have included the final function calls we would like you to run to train and evaluate your classifier. Feel free to re-use code you have already written or seen in previous notebooks!

In [1]:
# Download the wine .csv files from data archive
!rm -f winequality-red.csv winequality-white.csv
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv

--2025-07-13 17:19:18--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality-red.csv’

winequality-red.csv     [  <=>               ]  82.23K   222KB/s    in 0.4s    

2025-07-13 17:19:19 (222 KB/s) - ‘winequality-red.csv’ saved [84199]

--2025-07-13 17:19:19--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality-white.csv’

winequality-white.c     [   <=>              ] 258.23K   368KB/s    in 0.7s    

2025-07-13

In [2]:
# These are the packages required for this assignment
import pandas as pd
import numpy as np

# Use Pandas to read the csv file into a dataframe.
# Note that the delimiter in this csv is the semicolon ";" instead of a ,
df_red = pd.read_csv('winequality-red.csv',delimiter=";")

# Because we are performing a classification task, we will assign all red wine a label of 1
df_red["color"] = 1

# The method .head() is super useful for seeing a preview of our data!
df_red.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,1
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1


In [3]:
df_white = pd.read_csv('winequality-white.csv',delimiter=";")
df_white["color"] = 0  #assign white wine the label 0
df_white.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,0


In [4]:
# Now we combine our two dataframes
df = pd.concat([df_red, df_white])

# And shuffle them in place to mix the red and white wine data together
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color
0,6.6,0.22,0.29,14.40,0.046,39.0,118.0,0.99834,3.05,0.50,9.1,6,0
1,5.7,0.21,0.24,2.30,0.047,60.0,189.0,0.99500,3.65,0.72,10.1,6,0
2,6.1,0.19,0.37,2.60,0.041,24.0,99.0,0.99153,3.18,0.50,10.9,6,0
3,7.7,0.11,0.34,14.05,0.040,41.0,114.0,0.99634,3.07,0.59,11.0,7,0
4,7.5,0.21,0.29,1.50,0.046,35.0,107.0,0.99123,3.15,0.45,11.3,6,0


In [5]:
# We choose three attributes of the wine to perform our prediction on
input_columns = ["citric acid", "residual sugar", "total sulfur dioxide"]
output_columns = ["color"]

# We extract the relevant features into our X and Y numpy arrays
X = df[input_columns].to_numpy()
Y = df[output_columns].to_numpy()
print("Shape of X:", X.shape)
print("Shape of Y:", Y.shape)
in_features = X.shape[1]

Shape of X: (6497, 3)
Shape of Y: (6497, 1)


In [6]:
# Now we modify our implementation to create an abstract SingleNeuronModel
# class that will be the parent of our regression and classification models
class SingleNeuronModel():
    def __init__(self, in_features):
        #self.w = np.zeros(in_features)
        #self.w_0 = 0.
        # Better, we set initial weights to small normally distributed values.
        self.w = 0.01 * np.random.randn(in_features)
        self.w_0 = 0.01 * np.random.randn()
        self.non_zero_tolerance = 1e-8 # add this to divisions to ensure we don't divide by 0


    def forward(self, x):
        # Calculate and save the pre-activation z
        self.z = x @ self.w.T + self.w_0

        # Apply the activation function, and return
        self.a = self.activation(self.z)
        return self.a

    def activation(self, z):
        raise ImplementationError("activation method should be implemented by subclass")

    # calculate and save gradient of our output with respect to weights
    def gradient(self, x):
        raise ImplementationError("gradient method should be implemented by subclass")

    # update weights based on gradients and learning rate
    def update(self, grad_loss, learning_rate):
        model.w   -= grad_loss * self.grad_w   * learning_rate
        model.w_0 -= grad_loss * self.grad_w_0 * learning_rate

# New implementation! Single neuron classification model
class SingleNeuronClassificationModel(SingleNeuronModel):
    # Sigmoid activation function for classification
    def activation(self, z):
        return 1 / (1 + np.exp(-z) + self.non_zero_tolerance)

    # Gradient of output w.r.t. weights, for sigmoid activation
    def gradient(self, x):
        self.grad_w = self.a * (1-self.a) * x
        self.grad_w_0 = self.a * (1-self.a)

In [7]:
# Training process, using negative log likelihood (NLL) loss --
# appropriate for classification problems.

def train_model_NLL_loss(model, input_data, output_data,
                         learning_rate, num_epochs):
    non_zero_tolerance = 1e-8 # add this to the log calculations to ensure we don't take the log of 0
    num_samples = len(input_data)
    for epoch in range(1, num_epochs+1):
        total_loss = 0 #keep track of total loss across the data set

        for i in range(num_samples):
            x = input_data[i,...]
            y = output_data[i]
            y_predicted = model.forward(x)

            # NLL loss function
            loss = -(y * np.log(y_predicted + non_zero_tolerance) + (1-y) * np.log(1-y_predicted + non_zero_tolerance))
            total_loss += loss

            # gradient of prediction w.r.t. weights
            model.gradient(x)

            #gradient of loss w.r.t. prediction, for NLL
            grad_loss = (y_predicted - y)/(y_predicted * (1-y_predicted))

            # update our model based on gradients
            model.update(grad_loss, learning_rate)

        report_every = max(1, num_epochs // 10)
        if epoch == 1 or epoch % report_every == 0: #every few epochs, report
            print("epoch", epoch, "has total loss", total_loss)

In [8]:
# We will use this function to evaluate how well our trained classifier perfom
# Hint: the model you define above must have a .forward function in order to be compatible
# Hint: this evaluation function is identical to those in previous notebooks
def evaluate_classification_accuracy(model, input_data, labels):
    # Count the number of correctly classified samples given a set of weights
    correct = 0
    num_samples = len(input_data)
    for i in range(num_samples):
        x = input_data[i,...]
        y = labels[i]
        y_predicted = model.forward(x)
        label_predicted = 1 if y_predicted > 0.5 else 0
        if label_predicted == y:
            correct += 1
    accuracy = correct / num_samples
    print("Our model predicted", correct, "out of", num_samples,
          "correctly for", accuracy*100, "% accuracy")
    return accuracy

In [9]:
# Your Code Here!
model = SingleNeuronClassificationModel(in_features)

# train the model...
learning_rate = 0.001
epochs = 200

train_model_NLL_loss(model, X, Y, learning_rate, epochs)
print("\nFinal weights:")
print(model.w, model.w_0)

epoch 1 has total loss [6592.39636791]
epoch 20 has total loss [3545.56902575]
epoch 40 has total loss [3398.5383089]
epoch 60 has total loss [3369.42492839]
epoch 80 has total loss [3353.98048749]
epoch 100 has total loss [3352.2362953]
epoch 120 has total loss [3358.43705769]
epoch 140 has total loss [3360.41163557]
epoch 160 has total loss [3361.82224988]
epoch 180 has total loss [3361.14447762]
epoch 200 has total loss [3359.49961002]

Final weights:
[-0.87882295 -0.46516033 -0.17106265] [11.78600577]


In [11]:
evaluate_classification_accuracy(model, X, Y)

Our model predicted 5989 out of 6497 correctly for 92.18100661843927 % accuracy


0.9218100661843928